[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Signals and Systems (Signal- und Systemtheorie)

Summer Semester 2022 (Bachelor Course #24015)

- lecture: https://github.com/spatialaudio/signals-and-systems-lecture
- tutorial: https://github.com/spatialaudio/signals-and-systems-exercises

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import dimpulse, dstep, tf2zpk

# Exercise 9.3: 1-Pole System

A1D74A9E5B

In [ ]:
N = 6

coeff_a = -1/2  # 1-pole system
b = (1, 0)  # a and b should have same length
a = (1, coeff_a)  # make sure a[0]=1 !

# get zero/pole/gain
(z, p, g) = tf2zpk(b, a)
print(z, p, g)
print(np.allclose(p, - coeff_a))
print('b', b)  # b (1, 0)
print('a', a)  # a (1, -0.5)

## Impulse Response

In [ ]:
# for loop numeric solution
dirac = np.zeros(N)
dirac[0] = 1
y_old = 0  # init for k=-1
hnfor = np.zeros(N)
for k in range(N):  # start with k=0
    # not elegant, but to see what's going on
    y_new = b[0] * dirac[k] - a[1] * y_old  # recursion
    y_old = y_new  # set state for next k
    hnfor[k] = y_new  # store in array

# numeric solution
k, hn = dimpulse((b, a, 1), n=N)
hn = np.squeeze(hn)

# analytic solution
ha = (-coeff_a)**k

# plot
plt.figure()
plt.stem(k, hn)
plt.xlabel(r'$k$')
plt.ylabel(r'$h[k]$')
plt.title('impulse response')
plt.grid(True)

print(np.allclose(hnfor, ha))
print(np.allclose(hn, ha))

## Step Response

In [ ]:
# for loop numeric solution
stepsig = np.ones(N)
henfor = np.zeros(N)
y_old = 0  # init for k=-1
for k in range(N):  # start with k=0
    # not elegant, but to see what's going on
    y_new = b[0] * stepsig[k] - a[1] * y_old  # recursion
    y_old = y_new  # set state for next k
    henfor[k] = y_new  # store in array

# numeric solution
k, hen = dstep((b, a, 1), n=N)
hen = np.squeeze(hen)

# analytic solution
hea = 2 - (1/2)**k

# plot
plt.figure()
plt.stem(k, hen)
plt.xlabel(r'$k$')
plt.ylabel(r'$h_\epsilon[k]$')
plt.title('step response')
plt.grid(True)

print(np.allclose(henfor, hea))
print(np.allclose(hen, hea))

# Exercise 9.4: 2-Pole System

94A7A6D9E9

In [ ]:
g = 1

# z0 = 1/2 + 1j*np.sqrt(7)/2
# z1 = 1/2 - 1j*np.sqrt(7)/2
# ==
z0 = np.sqrt(2) * np.exp(+1j*np.arctan(np.sqrt(7)))
z1 = np.sqrt(2) * np.exp(-1j*np.arctan(np.sqrt(7)))

# p0 = 1/4 + 1j*np.sqrt(3)/4
# p1 = 1/4 - 1j*np.sqrt(3)/4
# ==
p0 = 1/2 * np.exp(+1j*np.pi/3)
p1 = 1/2 * np.exp(-1j*np.pi/3)

b = np.poly((z0, z1)) * g  # apply g here!
a = np.poly((p0, p1))
print('b', b)  # b [ 1. -1.  2.]
print('a', a)  # a [ 1.   -0.5   0.25]

In [ ]:
N = 8

# get zero/pole/gain
(z, p, g) = tf2zpk(b, a)
print(z, p, g)

# check zeros
print(np.allclose(np.angle(z[0]), np.angle(z0)))  # True here because g=1
print(np.allclose(np.angle(z[1]), np.angle(z1)))
print(np.allclose(np.abs(z[0]), np.abs(z0)))
print(np.allclose(np.abs(z[1]), np.abs(z1)))
# checl poles
print(np.allclose(np.angle(p[0]), np.angle(p0)))
print(np.allclose(np.angle(p[1]), np.angle(p1)))
print(np.allclose(np.abs(p[0]), np.abs(p0)))
print(np.allclose(np.abs(p[1]), np.abs(p1)))

## Impulse Response

In [ ]:
# for loop numeric solution
dirac = np.zeros(N)
dirac[0] = 1
y_state = np.zeros(3)  # init for k=-1, k=-2
x_state = np.zeros(3)  # init for k=-1, k=-2
hnfor = np.zeros(N)
for k in range(N):  # start with k=0
    # not elegant, but to see what's going on
    x_state[0] = dirac[k]
    fir_part = b[0] * x_state[0] + b[1] * x_state[1] + b[2] * x_state[2]
    iir_part = - a[1] * y_state[1] - a[2] * y_state[2]
    y_state[0] = fir_part + iir_part
    hnfor[k] = y_state[0]  # store in array
    x_state = np.roll(x_state, 1)  # roll into left is dont'care in next iter
    y_state = np.roll(y_state, 1)  # since we write x_state[0] and y_state[0]

# numeric solution
k, hn = dimpulse((b, a, 1), n=N)
hn = np.squeeze(hn)

# analytic solution
ha = 1/2**k * (-7*np.cos(np.pi/3*k) + 5/np.sqrt(3)*np.sin(np.pi/3*k))
ha[0] += 8

# plot
plt.figure()
plt.stem(k, hn)
plt.xlabel(r'$k$')
plt.ylabel(r'$h[k]$')
plt.title('impulse response')
plt.grid(True)

print(np.allclose(hnfor, ha))
print(np.allclose(hn, ha))

## Step Response

In [ ]:
# for loop numeric solution
stepsig = np.ones(N)
y_state = np.zeros(3)  # init for k=-1, k=-2
x_state = np.zeros(3)  # init for k=-1, k=-2
henfor = np.zeros(N)
for k in range(N):  # start with k=0
    # not elegant, but to see what's going on
    x_state[0] = stepsig[k]
    fir_part = b[0] * x_state[0] + b[1] * x_state[1] + b[2] * x_state[2]
    iir_part = - a[1] * y_state[1] - a[2] * y_state[2]
    y_state[0] = fir_part + iir_part
    henfor[k] = y_state[0]  # store in array
    x_state = np.roll(x_state, 1)  # roll into left is dont'care in next iter
    y_state = np.roll(y_state, 1)  # since we write x_state[0] and y_state[0]

# numeric solution
k, hen = dstep((b, a, 1), n=N)
hen = np.squeeze(hen)

# analytic solution
# TBD

# plot
plt.figure()
plt.stem(k, hen)
plt.xlabel(r'$k$')
plt.ylabel(r'$h_\epsilon[k]$')
plt.title('step response')
plt.grid(True)

print(np.allclose(henfor, hen))

## Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock* with
``github URL, commit number and/or version tag, year, (file name and/or content)``.